# =====================================
#  7A - L1 Introduction to tracking
# =====================================

## Examples of tracking



<br />
<figure>
  <img src="imgs/L71.gif" width=300/>
  <img src="imgs/L72.gif" width=300/>
  <center><figcaption>Fig.1(a) Old example</figcaption></center>
</figure>

------------------


<br />
<figure>
  <img src="imgs/L73.gif" width=300/>
  <img src="imgs/L74.gif" width=300/>
  <center><figcaption>Fig.1(b) Newer examples</figcaption></center>
</figure>

## Tracking Challenge

1. Many places it's hard to compute opticl flow.
2. There can be large displacements since could be moving rapidly
    probably need to take dynamics into account
3. Errors would compound - or drift
4. Occlusions, disocclusions 
    


## Shi-Tomasi feature tracker

"Only compute motion where you should?

Find good features using eigenvalues of second-moment matrix - you've seen the now twice!

- Key idea: "good" features to track are the ones that can be tracked reliably


1. From frame to frame, track with Lcas-Kanade and a pute translation model
    - More robust for small displacements, can be estimated from smaller neighborhoods
    
2. Check consistency of tracks by affine registration to the first (or earlier) observed instance of the feature
    - Affine model is more accurate for larger displacements
    - Comparing to the first or early frame helps to minimize drift
    
  <br />
<figure>
  <img src="imgs/L75.png" width=300/>
  <center><figcaption>Fig.2 J.Shi and C.Tomasi. *Good Features to Track* CVPR 1994</figcaption></center>
</figure>  

## Tracking with Dynamics

Key idea: Given a model of expected motion, predict where objects will occur in the next frame, even before seeing the image
 - Restrict search for object
 - Improved estimates since measurement noise is reduced by trajectory smoothness
 

## Detection vs Tracking

The idea of using prediction is the difference between tracking and just detecting


**Detection**: We *detect* the object *independently* in each frame

  <br />
<figure>
<img src="imgs/L76.png" width=500/>
<center><figcaption>Fig.3(a): detection</figcaption></center>
</figure> 

**Tracking**: We *predict* the new location of the object in the next frame using *estimated dynamics*. Then we *update* based upon measurements

   <br />
<figure>
<img src="imgs/L77.png" width=500/>
<center><figcaption>Fig.3(b): tracking</figcaption></center>
</figure> 

## Tracking with dynamics 

Key idea: Given a model of expected motion, predict where objects will occur in the next frame, even before seeing the image

**Goals**:

- Do less work looking fro the object, restrict the search.
- Get improved estimates since measurement noise is tempered by smoothness, dynamics priors

**Assumption** continuous (modeled) motion patterns:
- Objects do not disappear and reappear in different places in the scene  
- Camera is not moving instantaniously to new viewpoint
- Gradual change in pose between camera and scene
    
